In [ ]:
# import tensorflow as tf

# tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect()

# # instantiate a distribution strategy
# tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

In [ ]:
!pip install scikit-video

In [ ]:
from tensorflow import keras
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import numpy as np
import imageio
import cv2
import os
from tqdm import tqdm 
import skvideo.io
from glob import glob

df = pd.read_csv("/kaggle/input/heartdatabase/EchoNet-Dynamic/FileList.csv")
print(f"Total videos for training: {len(df)}")

df.head(10)

In [ ]:
path = '/kaggle/input/heartdatabase/EchoNet-Dynamic/Videos'

# # Import metadata about the echo images
# sequence_number = []
# train_img_w = []
# train_img_h = []
# for i in tqdm(os.listdir(path)[0:100]):
#     if i.endswith(".avi"):
        
#         number, width, height,_ = skvideo.io.vread(path +'/'+ i).shape
#         sequence_number.append(number)
#         train_img_w.append(width)
#         train_img_h.append(height)
        
# for a,b,c in zip(sequence_number,train_img_w,train_img_h):
#     print(a,b,c)

In [ ]:
# plt.hist(np.asarray(sequence_number), bins=10)
# plt.title("Distribution of the number of image across all patients in the training set")
# plt.xlabel("Number of images in the echo loop")
# plt.ylabel("Number of patients")
# plt.show()

In [ ]:
min(df.NumberOfFrames)

In [ ]:
train_df = df[df["Split"] == "TRAIN"]
train_files = df.FileName[df["Split"] == "TRAIN"]+'.avi'

train_df = train_df.reset_index(drop=True)
train_df

In [ ]:
train_files = list(train_files)


In [ ]:
val_df = df[df["Split"] == "VAL"]
val_files = df.FileName[df["Split"] == "VAL"]+'.avi'
val_df = val_df.reset_index(drop=True)
val_df.head()

In [ ]:
#val_files = list(val_files)
val_files

In [ ]:
test_df = df[df["Split"] == "TEST"]
test_files = df.FileName[df["Split"] == "TEST"]+'.avi'
test_df = test_df.reset_index(drop=True)
test_df

In [ ]:
test_files = list(test_files)
#test_files

In [ ]:
y = train_df.EF[np.where(train_df.FileName == '0X234005774F4CB5CD')[0][0]]
y

In [ ]:
# Because of large filesize we may need to import the image data into Python in batches. 
# This can be done using batch generators.

def batch_generator(batch_size, gen_x):
    batch_features = np.zeros((batch_size,28, 112, 112))
    batch_labels = np.zeros((batch_size,1))
    while True:
        for i in range(batch_size):
            batch_features[i] , batch_labels[i] = next(gen_x)
        yield np.expand_dims(batch_features,4), batch_labels

def generate_data(filelist, img_path, gt_df):
    while True:
        for i in filelist:
            if i.endswith(".avi"):
                #print(i[:-4])
                #img = np.load(img_path + i)
                img = skvideo.io.vread(img_path +'/'+ i)[:, :, :, 0] 
                img = img[:28]
                resized_img = np.zeros((28,112,112))
                for j,k in enumerate(img):
                    resized_img[j,:,:] = cv2.resize(k, (112,112), interpolation= cv2.INTER_LINEAR )/255
                y = round(gt_df.EF[np.where(gt_df.FileName == i[:-4])[0][0]])

                yield resized_img, y
                

In [ ]:
val_data =batch_generator(16,generate_data(val_files,path,val_df))
print("The shape of one batch of valid images (batch size = 16):")
print(next(val_data)[0].shape)

In [ ]:
for i in next(val_data)[0][0]:
    plt.imshow(i)
    plt.show()

In [ ]:
test = batch_generator(5,generate_data(test_files,'/kaggle/input/heartdatabase/EchoNet-Dynamic/Videos',test_df))

print("The shape of one batch of test images (batch size = 5):")
print(next(test)[0].shape)

In [ ]:
for i in next(test)[0][0]:
    plt.imshow(i)
    plt.show()

In [ ]:
# model = tf.keras.models.Sequential()
# model.add(
#     tf.keras.layers.TimeDistributed(
#         tf.keras.layers.Conv2D(32, (3,3), activation='relu'), 
#         input_shape=(28, 112, 112, 1) # 28 images...
#     )
# )
# model.add(
#     tf.keras.layers.TimeDistributed(
#         tf.keras.layers.GlobalAveragePooling2D() # Or Flatten()
#     )
# )
# model.add(
#     tf.keras.layers.LSTM(32, activation='relu', return_sequences=False)
# )
# model.add(tf.keras.layers.Dense(1))
# model.compile('adam', loss='mse', metrics=[tf.keras.metrics.RootMeanSquaredError()])

In [ ]:
# batch_size = 16
# num_epoch = 15
# steps = len(train_files[0:500])//batch_size
# print(steps)

# history = model.fit(x= batch_generator(batch_size,generate_data(train_files[0:500],path,train_df)),
#                     validation_data =(next(val_data)[0],next(val_data)[1]) ,
#                     epochs=num_epoch, steps_per_epoch=steps,
#                     verbose=1)

In [ ]:
from keras.models import Sequential
from keras.layers import Conv3D, MaxPooling3D, Flatten, Dense

model2 = Sequential()

# Add a 3D convolutional layer to process the video input
model2.add(Conv3D(32, kernel_size=(3, 3, 3), activation='relu', input_shape=(28, 112, 112, 1)))

# Add a 3D max pooling layer to reduce the spatial dimensions of the output
model2.add(MaxPooling3D(pool_size=(2, 2, 2)))

# Flatten the output from the previous layer to create a 1D feature vector
model2.add(Flatten())

# Add a fully connected layer with 64 units and ReLU activation
model2.add(Dense(64, activation='relu'))

# Add a final output layer with a single unit and linear activation to predict the ejection rate
model2.add(Dense(1, activation='linear'))

# Compile the model with the mean squared error loss and the Adam optimizer
model2.compile(loss='mean_squared_error', optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), metrics=[tf.keras.metrics.RootMeanSquaredError()])

callbacks = tf.keras.callbacks.ModelCheckpoint(
    'best.h5',
    monitor = "val_root_mean_squared_error",
    verbose= 1,
    save_best_only= True)

In [ ]:
batch_size = 32
num_epoch = 10
steps = len(train_files[0:500])//batch_size
print(steps)

history = model2.fit(x= batch_generator(batch_size,generate_data(train_files[0:500],path,train_df)),
                    validation_data =(next(val_data)[0],next(val_data)[1]) ,
                    epochs=num_epoch, steps_per_epoch=steps,callbacks=callbacks,
                    verbose=1)

In [ ]:
from tqdm import tqdm 

path = '/kaggle/input/heartdatabase/EchoNet-Dynamic/Videos'
best_model = tf.keras.models.load_model('best.h5')
y_pred=[]
y_true = []
for i in tqdm(sorted(os.listdir(path)[0:20])):
    if i.endswith(".avi"):
        #print(i[:-4])
        #img = np.load(img_path + i)
        img = skvideo.io.vread(path +'/'+ i)[:, :, :, 0] 
        img = img[:28]
        resized_img = np.zeros((28,112,112))
        for j,k in enumerate(img):
            resized_img[j,:,:] = cv2.resize(k, (112,112), interpolation= cv2.INTER_LINEAR )/255
        y_pred.append(best_model.predict(np.expand_dims(np.expand_dims(resized_img,3),0)))
        y_true.append(df.EF[np.where(df.FileName == i[:-4])[0][0]])

 
print(y_pred)
print()
print(y_true)

In [ ]:
# model.evaluate(test)

In [ ]:
rate = '15'
tmp_video = skvideo.io.vread('/kaggle/input/heartdatabase/EchoNet-Dynamic/Videos/0X3EAF0968B6971345.avi')
print(tmp_video.shape)

tmp_video = skvideo.io.vread('/kaggle/input/heartdatabase/EchoNet-Dynamic/Videos/0X3EAF0968B6971345.avi', outputdict={'-r': rate})
print(tmp_video.shape)

%matplotlib inline
from matplotlib import pyplot as plt
from matplotlib import animation
from IPython.display import HTML

# np array with shape (frames, height, width, channels)
video = tmp_video

fig = plt.figure()
im = plt.imshow(video[0,:,:,:])

plt.close() # this is required to not display the generated image

def init():
    im.set_data(video[0,:,:,:])
def animate(i):
    im.set_data(video[i,:,:,:])
    return im

anim = animation.FuncAnimation(fig, animate, init_func=init, frames=video.shape[0],interval=video.shape[0])
HTML(anim.to_html5_video())